In [3]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta as rd
import time
import math
from sklearn.metrics import mean_squared_error

from utils import *

In [4]:
class Argument(object):
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

argments = {
    'data_path': "../NCSOFT/financial_data",
    'result_path': "../results",
    'start_date': "2018-01-02",
    'end_date': "2018-12-31",
    }

args = Argument(**argments)

In [5]:
df_price, df_return, df_multifactor, start_date, end_date, start_year, end_year = read_data(args)

df_sp = pd.read_csv('./Index-tracking-portfolio-optimization-using-deep-learning/sp500.csv')
# df_sp = df_sp.set_index('Date')

In [6]:
class SNN(nn.Module):
    def __init__(self, input_dim, hidden_size, num_classes):
        super(SNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=0)
        
        self.gumbel_softmax = nn.functional.gumbel_softmax()
        
        
    def forward(self, w_tilde, S, tau):
        # Auxiliary Asset Selection Probabilities
        pi = np.exp(S/tau) / np.sum(np.exp(S/tau))
        
        # Gumbel-Max Trick #!
        z = self.gumbel_softmax(pi)
        
        # Element-wise exponentiation -> ensures positivity
        w_hat = np.exp(w_tilde)
        # Element-wise product with z -> allocates capital only to picked stocks
        w_bar = w_hat * z
        # Normalization -> ensures all capital is allocated
        w = w_bar / np.sum(w_bar)
        
        return w

    
        

In [13]:
stock_list = df_price.columns.values.tolist()
N = len(stock_list)
K = 10

print(N)

2480


In [14]:
w_tilde = torch.randn(N, 1, requires_grad=True)
S = torch.randn(K, N, requires_grad = True)

In [ ]:
learning_rate = 0.01
optimizer = torch.optim.SGD([w_tilde, S], lr=learning_rate)

epochs = 1000